# 오픈마켓 카테고리별 특성 파악

## 1. 데이터 수집(셀레니움 방식)

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import math
from tqdm.notebook import tqdm
from selenium import webdriver
import time
from datetime import datetime

In [ ]:
def 함수(soup,choice):    
    result = []
    경로 = []
    경로2 = []
    for link in soup.find_all('a',class_="itemcard__link"):
        경로.append(link.get('href'))
    for i in range(0,len(경로),2) :
        경로2.append(경로[i])
    for i in tqdm(range(choice)):
        이름 = soup.select('#bestContentsList > ul > li > div > div > div.itemcard__info__major > a > span > span.itemcard__title__name ' )[i].text
        가격 = soup.select('#bestContentsList > ul > li > div > div > div.itemcard__info__major > a > p > span > strong ' )[i].text
        구매 = soup.select('#bestContentsList > ul > li > div > div > div.itemcard__info__action > ul > li > span > b:nth-of-type(1)' )[i].text
        url2 = "https://www.g9.co.kr"+경로2[i]
        req = requests.get(url2)
        cont = req.content
        soup = BeautifulSoup(cont, 'html.parser')
        대분류 = soup.select('#container > div.vip_contents.vip_v3 > div.select_location > div:nth-of-type(1) > a')[0].text
        대분류 = 대분류.strip()
        중분류 = soup.select('#container > div.vip_contents.vip_v3 > div.select_location > div:nth-of-type(2) > a')[0].text
        중분류 = 중분류.strip()
        try:
            소분류 = soup.select('#container > div.vip_contents.vip_v3 > div.select_location > div:nth-of-type(3) > a')[0].text
            소분류 = 소분류.strip()
        except :
            소분류 = ''

        data = [이름,가격,구매,대분류,중분류,소분류]
        result.append(data)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
    return result

In [ ]:
driver = webdriver.Chrome(r'chromedriver.exe')
driver.implicitly_wait(3) # 잠시대기
driver.get('https://www.g9.co.kr/Display/Best')

while True :
    print("""카테고리 번호는 
전체 : 1, 해외직구: 2, 맛있는지구 : 3, 식품 : 4 뷰티 : 5
백화점.몰 : 6 , 홈스타일링 : 7, 주방.생필품 : 8, 가구.리빙.반려 : 9 , 가전 : 10 
컴퓨터 : 11 , 패션의류 : 12 , 신발.가방.주얼리 : 13 , 건강.자동차공구 : 14 , 출산.유아동 : 15
레저.아웃도어 : 16 , 취미.문구.도서 : 17 , e쿠폰 : 18 , 여행 : 19  , 패션편집샵 : 20
""")
    choice = int(input('원하는 숫자를 입력해주세요 : '))
    if 1 <= choice <=20:
        if choice >=15 :
            next1 = driver.find_element_by_xpath('//*[@id="plug-swipe-list1"]/button[2]')
            next1.click()
            number = driver.find_element_by_xpath(f'//*[@id="plug-swipe-list1"]/div/div/div[{choice}]/div/div/div/button')
            number.click()
            time.sleep(1)
        number = driver.find_element_by_xpath(f'//*[@id="plug-swipe-list1"]/div/div/div[{choice}]/div/div/div/button')
        number.click()            
        time.sleep(1)
        while True :
            choice = int(input('실시간: 0, 주간:1 선택해주세요'))
            if choice == 1:
                number = driver.find_element_by_xpath('//*[@id="contents"]/div[2]/ul/li[2]/button')
                number.click()
                break
            elif choice == 0:
                break
            print('다른 번호를 입력하셨습니다. ')
    else:
        print('다른번호를 입력하셨습니다.')
        continue           
    
    break
time.sleep(1)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
이름 = soup.select('#bestContentsList > ul > li > div > div > div.itemcard__info__major > a > span > span.itemcard__title__name ' )
print(f"선택한 페이지에서 상품 수는 {len(이름)}입니다. 몇개를 추출하시겠습니까?")
choice = int(input('고르기 : '))
result = 함수(soup,choice)
print("끝")

results_df = pd.DataFrame(result)
results_df.columns = ['title', 'price','구매','대','중','소']

today = datetime.today().strftime('%m.%d %H.%m')
results_df.to_csv(f'{today}.csv')
results_df

In [ ]:
#results_df.to_excel('./식품_실시간_0월00이.xlsx', index= False)

## 2. 데이터 전처리

In [ ]:
from pandas import DataFrame
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


### 파일 불러오기

In [ ]:
df1=pd.read_excel('./식품_실시간_1월22일.xlsx')
df2=pd.read_excel('./식품_주간_1월22일.xlsx')
'''
df1=pd.read_excel('./뷰티_실시간_1월22일.xlsx')
df2=pd.read_excel('./뷰티_주간_1월22일.xlsx')
'''
'''
df1=pd.read_excel('./홈스타일링_실시간_1월22일.xlsx')
df2=pd.read_excel('./홈스타일링_주간_1월22일.xlsx')
'''

### str을 int로 변환하기 1) ','제거 2) 형변환

In [ ]:
df1['구매']= df1['구매'].str.replace(',','')
df1['price']= df1['price'].str.replace(',','')
df2['구매']= df2['구매'].str.replace(',','')
df2['price']= df2['price'].str.replace(',','')

In [ ]:
df1['구매']=df1['구매'].astype(int)
df1['price']=df1['price'].astype(int)
df2['구매']=df2['구매'].astype(int)
df2['price']=df2['price'].astype(int)

### 랭킹, 매출량 칼럼 추가

In [ ]:
list = []
for i in range(1,101):
    list.append(i)
    
df1['랭킹'] = list
df2['랭킹'] = list

df1['매출량'] = df1['price'] * df1['구매']
df2['매출량'] = df2['price'] * df2['구매']

### 실시간, 주간 결합 테이블 d3 생성

In [ ]:
df11= df1[['title', '랭킹']]
df22 = df2[['title','랭킹']]
df3 = pd.merge(df11,df22 ,on = 'title', how='inner')
df3 = df3.rename({'랭킹_x':'랭킹(주간)', '랭킹_y':'랭킹(실시간)'}, axis='columns')

In [ ]:
df1

In [ ]:
df1_head = df1.head(10)
df1_head

In [ ]:
df2

In [ ]:
df2_head = df2.head(10)
df2_head

In [ ]:
df3

In [ ]:
df3.describe()

## 3. 데이터 시각화 (df1 : 실시간 df2 : 주간) *코드 default 주간

In [ ]:
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format
import matplotlib.pyplot as plt
# matplotlib 폰트설정
# plt.rc('font', family='NanumGothicOTF') # For MacOS
plt.rc('font', family='NanumGothic') # For Windows
print(plt.rcParams['font.family'])
import seaborn  as sns

In [ ]:
import platform
from matplotlib import font_manager, rc
import matplotlib.pyplot as plt
%matplotlib inline 
plt.rcParams['axes.unicode_minus'] = False
 
if platform.system() == 'Darwin':
    rc('font', family = 'AppleGothic')
elif platform.system() == 'Windows':
    path ='C:/Windows/Fonts/malgun.ttf'
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print("Can't find your system.")

In [ ]:
import platform
from matplotlib import font_manager, rc
import matplotlib.pyplot as plt
%matplotlib inline 
plt.rcParams['axes.unicode_minus'] = False
 
if platform.system() == 'Darwin':
    rc('font', family = 'AppleGothic')
elif platform.system() == 'Windows':
    path ='C:/Windows/Fonts/malgun.ttf'
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print("Can't find your system.")

In [ ]:
fig = plt.figure(figsize=(16,9))

### 중분류 groupby, 카테고리 중분류 비율

In [ ]:
df2.groupby('중').count()

In [ ]:
df2_중  = df2.groupby('중').count() #랭킹 100 안에 카테고리별  개수
df2_중 = df2_중[['title']]

In [ ]:
plt.figure(figsize=(18,15)) #원사이즈 조절 가능
plt.title('카테고리 중분류 비율')

plt.pie(df2_중['title'], labels = df2_중.index, autopct='%0.0f%%')
plt.show()

### Top 10 (df1_head or df2_head) 그래프

In [ ]:
sns.barplot(x= 'title', y='price' , data= df2_head)
plt.title('주간 카테고리 TOP 10(가격)')
plt.xticks(rotation =270)
plt.show()

In [ ]:
sns.barplot(x= 'title', y='구매' , data= df2_head)
plt.title('주간 카테고리 TOP 10(구매)')
plt.xticks(rotation =270)
plt.show()

In [ ]:
sns.barplot(x= 'title', y='매출량' , data= df2_head)
plt.title('주간 카테고리 TOP 10(매출량)')
plt.xticks(rotation =270)
plt.show()

In [ ]:
sns.boxplot(x = "중", y = "구매", data = df2)
plt.title('주간 카테고리 TOP 10 Box Plot')

plt.xticks(rotation =270)
plt.show()